In [1]:
import pandas as pd
import numpy as np
from flask import Flask, jsonify, request
from tensorflow.python.keras.models import load_model
import pickle
from tensorflow.python.keras.models import model_from_json
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.models import load_model
import tensorflow as tf
from tensorflow.python.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers import GRU, Input, Dense, Flatten, Dropout, BatchNormalization, Embedding
from tensorflow.python.keras.models import Sequential, K
import seaborn as sns
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score, roc_curve, accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
from numpy import array
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")


gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_virtual_device_configuration(
    gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1750)])
if 'session' in locals() and session is not None:
    print('Close interactive session')
    session.close()

# load model
#model = pickle.load(open('model.pkl','rb'))
model = load_model('new_model2same.h5')

# data_path = ['data']
# filepath = os.sep.join(data_path + ['result_nodup.csv'])     
# result_nodup = pd.read_csv(filepath, sep=',', engine='python')
# result_nodup = result_nodup.drop(columns=['Unnamed: 0'])


accepted_chars = 'abcdefghijklmnopqrstuvwxyzàáâãäåçèéêëìíîïñòóôõöøùúüýÿčńōŕřšūž-–'
word_vec_length = 25 
char_vec_length = 63
output_labels = 2 
char_to_int = dict((c, i) for i, c in enumerate(accepted_chars))
int_to_char = dict((i, c) for i, c in enumerate(accepted_chars))

# Removes all non accepted characters
def normalize(line):
    return [c.lower() for c in line if c.lower() in accepted_chars]

# Returns a list of n lists with n = word_vec_length
def name_encoding(name):
     # Encode input data to int, e.g. a->1, z->26
    integer_encoded = [char_to_int[char] for i, char in enumerate(name) if i < word_vec_length]
    
    # Start one-hot-encoding
    onehot_encoded = list()
    
    for value in integer_encoded:
        # create a list of n zeros, where n is equal to the number of accepted characters
        letter = [0 for _ in range(char_vec_length)]
        letter[value] = 1
        onehot_encoded.append(letter)
        
    # Fill up list to the max length. Lists need do have equal length to be able to convert it into an array
    for _ in range(word_vec_length - len(name)):
        onehot_encoded.append([0 for _ in range(char_vec_length)])
        
    return onehot_encoded




In [2]:
namecheck = [input("Name or Surname: ")]
print(namecheck)
test = np.asarray([np.asarray(name_encoding(normalize(name)))
                   for name in namecheck]).astype(np.float32)
pred = model.predict(np.asarray(test))
predictoin = np.squeeze(pred * 100)
temp = ''.join(namecheck)
if predictoin > 50:
    print("I am", predictoin, "sure", '"{}"'.format(temp), "is a name")
else:
    print("I am", (100 - predictoin), '"{}"'.format(temp), "is a surname")

Name or Surname: Aleknsar
['Aleknsar']
I am 53.300125 sure "Aleknsar" is a name
